# Preparation of Flux-Free Kitaev Honeycomb lattice state
## In HONEYCOMB LATTICE SPACE
2 honeycombs

N = 14 spins

Hamiltonian: $H = -J_x \sum_{\langle i,j \rangle_x} \sigma_i^x \sigma_j^x
    -J_y \sum_{\langle i,j \rangle_y} \sigma_i^y \sigma_j^y
    -J_z \sum_{\langle i,j \rangle_z} \sigma_i^z \sigma_j^z$ 

Step 1: prepare product state $|00..>$ of N spins

In [1]:
import numpy as np
import scipy
from scipy import sparse

We are doing this mapping: 

$\ket{\uparrow \uparrow} = \ket{00} , \quad\ket{\downarrow \downarrow} = \ket{11} $

<div align="center">
    <img src="Honeycomb_Mapping.jpeg" alt="Honeycomb Mapping" width="400"/>
</div>


In [2]:
n_spins = 14
e1 = sparse.csr_array([[1],[0]])
psi = e1
for _ in range(n_spins-1):
    psi = sparse.kron(psi,e1, format='csr')

print(psi.shape)


assert psi.shape[0]== 2**14

(16384, 1)


In [3]:
print(psi)

  (0, 0)	1


Identify a representative qubit in each plaquette: we identify qubit 0 and 6

We create operator that applies hadamard only on these two representative qubits

In [4]:
def Hadamard(qubit1, qubit2, n):
    assert qubit1 < qubit2
    Id = sparse.csr_array(np.eye(4))
    H_small = sparse.csr_array([[1./np.sqrt(2), 0., 0., 1./np.sqrt(2)], [0., 1., 0., 0.], [0., 0., 1., 0.], [1./np.sqrt(2), 0., 0., -1./np.sqrt(2)]])
    op_list = [Id]*n
    op_list[qubit1] = H_small
    op_list[qubit2] = H_small
    
    H = op_list[0]

    for op in op_list[1:]:
        H = sparse.kron(H,op, format='csr')
    return H
    

In [6]:
qubit1 = 0
qubit2 = 4
n_eff_spins = int(n_spins/2)
H = Hadamard(qubit1,qubit2,n_eff_spins)
print(H.shape)
psi1 = H @ psi.copy()

print(psi1.shape)
print(psi1)

(16384, 16384)
(16384, 1)
  (0, 0)	0.4999999999999999
  (48, 0)	0.4999999999999999
  (12288, 0)	0.4999999999999999
  (12336, 0)	0.4999999999999999


Create op. that applies CNOT on each plaquette with representative qubits as control qubits and all other qubits of plaquette as target ones

In [7]:
def CNOT(qubit1, qubit2, n, l): #l = effective spins per plaquette, n = tot EFFECTIVE spins
    assert qubit1 < 3
    assert 3 < qubit2
    # Id = sparse.csr_array(np.eye(2))
    # X = sparse.csr_array([[0.,1.],[1.,0.]])
    # Proj_0 = sparse.csr_array([[1.,0.],[0.,0.]]) #|0><0|
    # Proj_1 = sparse.csr_array([[0.,0.],[0.,1.]]) #|1><1|
    
    newId = sparse.csr_array(np.eye(2**(2*n-2*l)))
    CNOT_p1 = sparse.kron(plaquetteCNOT(qubit1,l),newId,'csr')
    CNOT_p2 = sparse.kron(newId,plaquetteCNOT(qubit2-l+1,l), 'csr') #we map qubit 3 -> 0, 4->1, 5->2, 6->3
    # even if geometry is not conserved, it still works because CNOT applied on this 
    #sequence of hilbert spaces in this order does not depend on geometry!
    # print(CNOT_p2 @ CNOT_p1)
    # print(CNOT_p1 @ CNOT_p2)
    assert np.array_equal((CNOT_p2 @ CNOT_p1).toarray(), (CNOT_p1 @ CNOT_p2).toarray()), "Sparse arrays are not equal"
    return CNOT_p2 @ CNOT_p1

def plaquetteCNOT(qubit, l):
    Id = sparse.csr_array(np.eye(4))
    X = np.zeros((4,4))
    X[0,3] = 1.
    X[3,0] = 1.
    X = sparse.csr_array(X)
    #print(X)
    
    Proj_0 = np.zeros((4,4)) #|00><00|
    Proj_0[0,0] = 1
    Proj_0 = sparse.csr_array(Proj_0)
    Proj_1 = np.zeros((4,4)) #|11><11|
    Proj_1[3,3] = 1
    Proj_1 = sparse.csr_array(Proj_1)

    op_list_id = [Id]*l
    op_list_id[qubit] = Proj_0

    op_list_x = [X]*l
    op_list_x[qubit] = Proj_1

    CNOT1 = op_list_id[0]
    CNOT2 = op_list_x[0]

    for op1, op2 in zip(op_list_id[1:],op_list_x[1:]):
        CNOT1 = sparse.kron(CNOT1, op1, 'csr')
        CNOT2 = sparse.kron(CNOT2, op2, 'csr')
    return CNOT1 + CNOT2

In [20]:
CNOT_op = CNOT(qubit1, qubit2, n_eff_spins, 4)
#print(CNOT_op.toarray())
psi2 =  CNOT_op @ psi1.copy()
#print(psi1.shape)
#print(CNOT_op.shape)
#print(psi2.shape)
print(psi2)

  (0, 0)	0.4999999999999999
  (255, 0)	0.4999999999999999
  (16191, 0)	0.4999999999999999
  (16320, 0)	0.4999999999999999


Next step: apply $U_a$ to horizontal sites: effective qubit = 0,2,4,5, $U_b$ to vertical ones: effective qubit =1,3,6  $U = \prod_{vert} U_a \prod_{hor} U_b$

In [21]:
def U_a_full(n_eff_spins):
    U_a = sparse.csr_array([[(1.+1.j)/2., 0., 0., (1.+1.j)/2.],[0., 1., 0., 0.], [0., 0., 1., 0.], [(-1.+1.j)/2., 0., 0., (1.-1.j)/2.]])
    Id = sparse.csr_array(np.eye(4))
    op_list = [U_a]*n_eff_spins
    op_list[1] = Id
    op_list[3] = Id
    op_list[6] = Id

    U = op_list[0]

    for op in op_list[1:]:
        U = sparse.kron(U,op, format='csr')
    return U

def U_b_full(n_eff_spins):
    U_b = sparse.csr_array([[1./np.sqrt(2)*(1.-1.j),0., 0., 0.],[0., 1., 0., 0.], [0., 0., 1., 0.], [0., 0., 0., 1./np.sqrt(2)*(1.+1.j)]])
    Id = sparse.csr_array(np.eye(4))
    op_list = [Id]*n_eff_spins
    op_list[1] = U_b
    op_list[3] = U_b
    op_list[6] = U_b

    U = op_list[0]

    for op in op_list[1:]:
        U = sparse.kron(U,op, format='csr')
    return U


In [22]:
psi_final = U_a_full(n_eff_spins) @ U_b_full(n_eff_spins) @ psi2.copy()
print(psi_final.shape)
print(psi_final)

(16384, 1)
  (0, 0)	(0.0883883476483184+0.0883883476483184j)
  (12, 0)	(-0.0883883476483184+0.0883883476483184j)
  (48, 0)	(-0.0883883476483184+0.0883883476483184j)
  (60, 0)	(-0.0883883476483184-0.0883883476483184j)
  (195, 0)	(-0.08838834764831839-0.0883883476483184j)
  (207, 0)	(-0.0883883476483184+0.08838834764831839j)
  (243, 0)	(-0.0883883476483184+0.08838834764831839j)
  (255, 0)	(0.08838834764831839+0.0883883476483184j)
  (768, 0)	(-0.0883883476483184+0.0883883476483184j)
  (780, 0)	(-0.0883883476483184-0.0883883476483184j)
  (816, 0)	(-0.0883883476483184-0.0883883476483184j)
  (828, 0)	(0.0883883476483184-0.0883883476483184j)
  (963, 0)	(0.0883883476483184-0.08838834764831839j)
  (975, 0)	(-0.08838834764831839-0.0883883476483184j)
  (1011, 0)	(-0.08838834764831839-0.0883883476483184j)
  (1023, 0)	(-0.0883883476483184+0.08838834764831839j)
  (3075, 0)	(-0.0883883476483184-0.08838834764831839j)
  (3087, 0)	(-0.08838834764831839+0.0883883476483184j)
  (3123, 0)	(-0.08838834764831

In [23]:
# Id = sparse.csr_array(np.eye(4))

# X = np.zeros((4,4))
# X[0,3] = 1.
# X[3,0] = 1.
# X = sparse.csr_array(X)

# Z = np.zeros((4,4))
# Z[0,0] = 1.
# Z[3,3] = -1.
# Z = sparse.csr_array(Z)

# Y = np.zeros((4,4))
# Y[0,1] = -1.j
# Y[1,0] = 1.j
# Y[2,3] = -1.j
# Y[3,2] = 1.j
# Y = sparse.csr_array(Y)

X = sparse.csr_array([[0.,1.],[1.,0.]])
Y = sparse.csr_array([[0.,-1.j],[1.j,0.]])
Z = sparse.csr_array([[1.,0.],[0.,-1.]])
I = sparse.csr_array(np.eye(2))

Now we create plaquette term!

In [24]:
def Op_full(O, Id, pos, n):
    op_list = [Id]*n

    if isinstance(pos, list):
        for p in pos:
            op_list[p] = O
    else:
        op_list[pos] = O

    fullOp = op_list[0]
    for op in op_list[1:]:
        fullOp = sparse.kron(fullOp,op, format='csr')
    return fullOp   

In [25]:
def W_p(Op_list, index_list, Id, n_spins):
    assert len(Op_list) == len(index_list)
    assert len(Op_list) > 0
    assert len(index_list) > 0
    full_op = []
    for op, idx in zip(Op_list, index_list):
        full_op.append(Op_full(op, Id, idx, n_spins))
    W = full_op[0]
    for op in full_op[1:]:
        W = W @ op
    return W

In [26]:
Op_list = [X, Y, Z, X, Y, Z]
index_list = [3, 2, 1, 6, 7, 4]

W_tilde_1 = W_p(Op_list, index_list, I, n_spins)

# Compute the complex conjugate transpose of psi_final
psi_final_dagger = psi_final.conjugate().transpose()

# Compute the expectation value
expectation_value = (psi_final_dagger @ W_tilde_1 @ psi_final)[0,0]

# Print the result
print("First plaquette term:", expectation_value)

First plaquette term: (0.9999999999999998+0j)


In [27]:
Op_list = [X, Y, Z, X, Y, Z]
index_list = [7, 6, 9, 12, 13, 10]

W_tilde_2 = W_p(Op_list, index_list, I, n_spins)

# Compute the expectation value
expectation_value = (psi_final_dagger @ W_tilde_2 @ psi_final)[0,0]

# Print the result
print("Second plaquette term:", expectation_value)

Second plaquette term: (0.9999999999999999+0j)


We finally get the plaquette terms to be +1! So now we can apply a floquet drive to the flux free state. Note that the plaquette term commutes with the floquet time evolution, which means that it is a conserved quantity!

Before time evolving, let's do one last check. We check that the initial flux free state is an eigenstate of the ZZ terms on the vertical bonds. This means that the fermions are aligned on the vertical bonds, with one Majorana per site.

In [70]:
ZZ_1 = Op_full(Z, I, [2,3], n_spins)
ZZ_2 = Op_full(Z, I, [6,7], n_spins)
ZZ_3 = Op_full(Z, I, [12,13], n_spins)
ZZ_other1 = Op_full(Z, I, [0,1], n_spins)
ZZ_other2 = Op_full(Z, I, [4,5], n_spins)

ZZ1Value = (psi_final_dagger @ ZZ_1 @ psi_final)[0,0]
ZZ2Value = (psi_final_dagger @ ZZ_2 @ psi_final)[0,0]
ZZ3Value = (psi_final_dagger @ ZZ_3 @ psi_final)[0,0]
ZZother1Value = (psi_final_dagger @ ZZ_other1 @ psi_final)[0,0]
ZZother2Value = (psi_final_dagger @ ZZ_other2 @ psi_final)[0,0]

print("ZZ1Value:", ZZ1Value)
print("ZZ2Value:", ZZ2Value)
print("ZZ3Value:", ZZ3Value)
print("ZZother1Value:", ZZother1Value)
print("ZZother2Value:", ZZother2Value)

ZZ1Value: (0.9999999999999999+0j)
ZZ2Value: (0.9999999999999999+0j)
ZZ3Value: (0.9999999999999999+0j)
ZZother1Value: (0.9999999999999999+0j)
ZZother2Value: (0.9999999999999999+0j)


### Floquet drive

Ops $e^{i \frac{\pi}{4} \alpha \alpha} $ with $\alpha = X, Y, Z$

In [ ]:
# X_terms = []
# Xindexlist = [[1,2],[4,7],[6,9],[10,13]]

# for indeces in Xindexlist:
#     X_terms.append(expm(1.j*np.pi/4* Op_full(X, I, indeces, n_spins)))
    
#this method take too much time to construct the full unitary operator!



/Users/Giovanni/miniconda3/envs/Computational/lib/python3.12/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:603: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
/Users/Giovanni/miniconda3/envs/Computational/lib/python3.12/site-packages/scipy/sparse/linalg/_matfuncs.py:707: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  return spsolve(Q, P)


KeyboardInterrupt: 

In [73]:
from scipy.sparse.linalg import expm_multiply


In [ ]:
Xindexlist = [[1,2],[4,7],[6,9],[10,13]]
Yindexlist = [[1,6],[3,4],[9,12],[7,10]]

def floquet_unitary(psi, n_spins, X = X, Z = Z, Y = Y, I = I, 
                    applyX = True, applyY = True, applyZ = True,
                    Xindexlist = Xindexlist, Yindexlist = Yindexlist):
    #apply XX unitary

    psi_t = psi

    i = 0
    j = 0
    z = 0

    #maybe do first sum of sparse matrices XX and then call one unique expm_mulitply

    if applyX:
        psi_t = expm_multiply(+1.j * np.pi/4. * Op_full(X, I, Xindexlist[0], n_spins), psi_t)

        for indeces in Xindexlist[1:]:
            psi_t = expm_multiply(+1.j * np.pi/4. * Op_full(X, I, indeces, n_spins), psi_t)

        i =1

    #apply YY unitary

    if applyY:
        psi_t = expm_multiply(+1.j * np.pi/4. * Op_full(Y, I, Yindexlist[0], n_spins), psi_t)

        for indeces in Yindexlist[1:]:
            psi_t = expm_multiply(+1.j * np.pi/4. * Op_full(Y, I, indeces, n_spins), psi_t)
        j = 1

    #apply ZZ unitary: check
    if applyZ:
        for n in range(0, n_spins, 2):
            psi_t = Op_full(expm(+1.j * np.pi/4. * sparse.kron(Z,Z, 'csr')), I, n, n_spins-1) @ psi_t
        z = 1
    
    print("i,j,z", i,j,z)

    return psi_t

def floquet_evolution(psi, n_cycles, n_spins, X = X, Z = Z, Y = Y, I = I, ZZ = ZZ_2, 
                      applyX = True, applyY = True, applyZ = True,
                      Xindexlist = Xindexlist, Yindexlist = Yindexlist):
    # Apply the Floquet unitary operator to the initial state
    list_zz = []
    psi_dagger = psi.conjugate().transpose()
    list_zz.append((psi_dagger @ ZZ @ psi)[0,0])
    for _ in range(n_cycles):
        psi = floquet_unitary(psi, n_spins, X, Z, Y, I)
        psi_dagger = psi.conjugate().transpose()
        list_zz.append((psi_dagger @ ZZ @ psi)[0,0])

    return psi, list_zz


In [84]:
psi_t, _ = floquet_evolution(psi_final, 1, n_spins)
print(psi_t.shape)
#print(psi_t)

/Users/Giovanni/miniconda3/envs/Computational/lib/python3.12/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:603: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
/Users/Giovanni/miniconda3/envs/Computational/lib/python3.12/site-packages/scipy/sparse/linalg/_matfuncs.py:707: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  return spsolve(Q, P)


i,j,z 1 1 1
(16384, 1)


Let us now recheck the plaquette terms! (they should still be +1)

In [85]:
# Compute the complex conjugate transpose of psi_final
psi_t_dagger = psi_t.conjugate().transpose()

# Compute the expectation value 1
expectation_value1 = (psi_t_dagger @ W_tilde_1 @ psi_t)[0,0]

# Print the result
print("First plaquette term:", expectation_value1)


# Compute the expectation value 2
expectation_value2 = (psi_t_dagger @ W_tilde_2 @ psi_t)[0,0]

# Print the result
print("Second plaquette term:", expectation_value2)

First plaquette term: (0.9999999999999999+2.6481536735324493e-34j)
Second plaquette term: (0.9999999999999999+3.3703774026776627e-34j)


Now let us check how the fermion on the central vertical bond changes over floquet drive, i.e. let us calculate the expectation value ZZ on the central vertical bond (spins 6,7) and see how it changes. we expect it to oscillate between +1 and 0 per floquet cycle 

In [77]:
n_cycles = 5

psinew, list_zz = floquet_evolution(psi_final, n_cycles, n_spins)
    
print(list_zz)

[(0.9999999999999999+0j), (4.553649124439119e-18+0j), (1+0j), (6.245004513516506e-17+0j), (1+0j), 0j]


Now we go back to psi_final, which is a flux free state with fermions paired vertically, and we want to rearrange them such that they are paired diagonally (fig. 4b thesis paper https://arxiv.org/pdf/2501.18461). We do so by applying the YY evolution to the sites on the low part of the hexagon connected by the y-y bonds, and then the XX evolution to the "low" sites connected by XX bonds, as shown in figure

 ![Image](M-Swap_gates.png)

$\textbf{Why can't we drive the YY-XX also in the upper part of the honeycomb}$?

We don't apply the evolutions to also the upper sites otherwise instead of having diagonal fermions weird things happen, you can convince yourself by doing the diagrams by hand on paper

In [ ]:
psi_diagonal1 = floquet_unitary(psi_final, n_spins, applyX = False, applyY = True, applyZ = False, Yindexlist = [[3,4],[7,10]])
psi_diagonal2 = psi_diagonal1.copy()
psi_diagonal2 = floquet_unitary(psi_diagonal2, n_spins, applyX = True, applyY = False, applyZ = False, Xindexlist = [[4,7],[10,13]])
psi_diagonal = psi_diagonal2.copy()

i,j,z 0 1 0
i,j,z 1 0 0


Now let us check if this state is eigenstate with eigval = +1 of the ZZ operators acting on the [2,7], and [6,13] sites 

In [87]:
ZXZX = W_p([Z, X, Z, X], [2, 3, 4, 7], I, n_spins)

# Compute the complex conjugate transpose of psi_diagonal
psi_diagonal_dagger = psi_diagonal.conjugate().transpose()

ZXZX_Value = (psi_diagonal_dagger @ ZXZX @ psi_diagonal)[0,0]

print("ZXZX Value:", ZXZX_Value)


ZXZX Value: (0.9999999999999999+0j)


### e-m anyons

Now we want to create an e anyon at the center of the first cell. To do so we have to apply a Z operator to the first plaquette with already the diagonal fermion, in order to have plaquette term Wp = -1 and a diagonal fermion!